In [34]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeRegressor

df = pd.read_csv("GroupExam2024_data_and_documentation/elektronisk-rapportering-ers-2018-fangstmelding-dca-simple.csv", sep=';', decimal=',')

### Decision tree modell:
- **Mål:** 
Utforske datasettet og prøve ut DecisionTreeRegressor.
- **Bakgrunn:** 
Har utforsket havforskningsinstituttet sine kart og datatjenester og har gjort noen observasjoner angående fiske i havområdene rundt Norge.

- **Tanker omkring forarbeid:**
Ut fra karttjenestene til havforskningsinstituttet, ser vi at det foregår et omfattende sei-fiske - spesielt i områdene vest for Norge. Torskefiske, hvor torsk er hovedart foregår først og fremst i lofoten (uten trål) og i de nordligste områdene, kanskje spesielt rundt Bjørnøya.
Ellers ser vi at de mest brukte fiskeredskapene er Bunntrål, Snurrevad og Andre liner. Bunntrål og snurrevad er relativt like redskaper i natur, så vi kunne vurdert å bruke modellen med begge disse redskapene. Valget faller likevel på bunntrål, siden det er mest representert i datasettet.

Områdene vi har valgt å analysere er 8, 42, 28, og 7 som er områder vest for Bergen.
Her er det hovedsaklig fiske etter Sei, og det er da naturlig å undersøke dette fiske, da det er nærliggende å anta at fiske etter ulik type fisk foregår i ulik dybde, lokasjoner, tidspunkt osv.

In [35]:
# Aktuelle redskaper å bruke: Bunntrål, Snurrevad og muligens Andre Liner. evt, dobbeltål
# Unngå garn.
# Aktuelle fisketyper å bruke hovedfangst: Sei
fish_types = ['Sei']
# Aktuelle Redskaper: Bunntrål, Snurrevad, Andre liner, (Dobbeltrål)
tools = ['Bunntrål']  # , 'Snurrevad', 'Andre liner'
# Aktuelle områder:
#areas = [x for x in range(0, 70)] # alle hovedområder minus antarktis og fiske i andre hav.
areas = [8, 42, 28, 7]

condition_3 = df['Hovedområde start (kode)'].isin(areas)
condition_1 = df['Hovedart FAO'].isin(fish_types) 
condition_2 = df['Redskap FDIR'].isin(tools)
combined_condition = condition_1 & condition_2 & condition_3
filtered_df = df[combined_condition].copy()
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21253 entries, 90 to 305299
Data columns (total 45 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Melding ID                21253 non-null  int64  
 1   Meldingstidspunkt         21253 non-null  object 
 2   Meldingsdato              21253 non-null  object 
 3   Meldingsklokkeslett       21253 non-null  object 
 4   Starttidspunkt            21253 non-null  object 
 5   Startdato                 21253 non-null  object 
 6   Startklokkeslett          21253 non-null  object 
 7   Startposisjon bredde      21253 non-null  float64
 8   Startposisjon lengde      21253 non-null  float64
 9   Hovedområde start (kode)  21253 non-null  float64
 10  Hovedområde start         21253 non-null  object 
 11  Lokasjon start (kode)     21253 non-null  float64
 12  Havdybde start            21253 non-null  int64  
 13  Stopptidspunkt            21253 non-null  object 
 14  Stoppdato

In [36]:
filtered_df.loc[:, 'Bruttotonnasje kombinert'] = filtered_df['Bruttotonnasje 1969'].fillna(filtered_df['Bruttotonnasje annen'])

for start_stop in ['Startklokkeslett', 'Stoppklokkeslett']:
    # Konvertere klokkeslett til relevant verdi
    filtered_df[start_stop] = pd.to_datetime(filtered_df[start_stop], format='%H:%M').dt.strftime('%H-%M')
    # Konverter til Desimal-timer.
    filtered_df[f'{start_stop}_DecimalHours'] = filtered_df[start_stop].str.split('-').apply(lambda x: int(x[0]) + int(x[1]) / 60)
    # Konverter til Radianer.
    filtered_df[f'{start_stop}_Radians'] = filtered_df[f'{start_stop}_DecimalHours'] * (2 * np.pi / 24)
    # Regne ut sinus og cosinus.
    filtered_df[f'{start_stop}_TimeSin'] = np.sin(filtered_df[f'{start_stop}_Radians'])
    filtered_df[f'{start_stop}_TimeCos'] = np.cos(filtered_df[f'{start_stop}_Radians'])

In [37]:
filtered_df = filtered_df.drop(columns = [
    'Fangstår', 
    #'Melding ID', 
    'Meldingstidspunkt', 
    'Meldingsdato',
    'Meldingsklokkeslett', 
    'Starttidspunkt',
    'Startklokkeslett',
    #'Hovedområde start (kode)', 
    'Hovedområde start',
    'Lokasjon start (kode)', 
    'Havdybde start', 
    'Stopptidspunkt',
    'Varighet',
    'Stoppdato', 
    'Startposisjon bredde',
    'Startposisjon bredde',
    #'Stoppklokkeslett', 
    #'Stopposisjon bredde', 
    #'Stopposisjon lengde',
    'Hovedområde stopp (kode)', 
    'Hovedområde stopp',
    'Lokasjon stopp (kode)', 
    'Redskap FAO (kode)', 
    'Redskap FDIR (kode)',
    'Hovedart FAO (kode)',
    'Hovedart - FDIR (kode)', 
    'Art FAO (kode)', 
    #'Art FAO',
    'Art - FDIR (kode)', 
    'Art - FDIR', 
    'Art - gruppe (kode)',
    #'Art - gruppe', 
    'Lengdegruppe (kode)', 
    'Lengdegruppe',
    'Bredde',
    'Fartøylengde',
    'Bruttotonnasje 1969',
    'Bruttotonnasje annen'
    ])

In [38]:
def join_unique_values(series):
    return ', '.join(series.dropna().astype(str).unique())

# Gruppere df og slå sammen rader som kommer av samme aktivitet:
grouped_df = filtered_df.groupby(['Melding ID', 'Stoppklokkeslett'], as_index= False).agg({
    'Stopposisjon lengde': 'first',
    'Stopposisjon bredde': 'first',
    'Stoppklokkeslett': 'first',
    'Stoppklokkeslett_DecimalHours': 'first',
    'Stoppklokkeslett_Radians': 'first',
    'Stoppklokkeslett_TimeSin': 'first',
    'Hovedområde start (kode)': 'first',
    'Havdybde stopp': 'last',
    'Trekkavstand': 'first',  
    'Redskap FDIR': join_unique_values,
    'Hovedart FAO': 'first',  
    'Art FAO': join_unique_values,
    'Art - gruppe': join_unique_values,
    'Rundvekt': 'sum',   
    'Bruttotonnasje kombinert': 'first'
})


In [39]:
# Droppe tomme rader.
cleaned_df = grouped_df.copy().dropna()
# Konvertere positive Havdybdeverdier
cleaned_df['Havdybde stopp'] = cleaned_df['Havdybde stopp'].abs()
# Ta vekk havdybder på over 10 m, for å fjerne usannsynlige tråledybder.
cleaned_df = cleaned_df[cleaned_df['Havdybde stopp'] > 10]
# Droppe trekkavstand på over ei viss lengd.
cleaned_df = cleaned_df[(cleaned_df['Trekkavstand'] > 10) & (cleaned_df['Trekkavstand'] < 60000)]

# Droppe ekstreme Bruttotonnasjeverdier. Under
cleaned_df = cleaned_df[(cleaned_df['Bruttotonnasje kombinert'] > 400) & (cleaned_df['Bruttotonnasje kombinert'] < 4000)]
# Droppe registreringer i områder som ikkje er i Norsk farvatn. Startposisjon med mindre enn 0.
cleaned_df = cleaned_df[cleaned_df['Stopposisjon lengde'] > 0]
cleaned_df = cleaned_df[cleaned_df['Stopposisjon bredde'] > 0]
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3889 entries, 0 to 5741
Data columns (total 16 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Melding ID                     3889 non-null   int64  
 1   Stopposisjon lengde            3889 non-null   float64
 2   Stopposisjon bredde            3889 non-null   float64
 3   Stoppklokkeslett               3889 non-null   object 
 4   Stoppklokkeslett_DecimalHours  3889 non-null   float64
 5   Stoppklokkeslett_Radians       3889 non-null   float64
 6   Stoppklokkeslett_TimeSin       3889 non-null   float64
 7   Hovedområde start (kode)       3889 non-null   float64
 8   Havdybde stopp                 3889 non-null   int64  
 9   Trekkavstand                   3889 non-null   float64
 10  Redskap FDIR                   3889 non-null   object 
 11  Hovedart FAO                   3889 non-null   object 
 12  Art FAO                        3889 non-null   object

In [40]:

# Lager en liste med features
all_features = [
    'Stopposisjon lengde', 
    'Stopposisjon bredde', 
    'Stoppklokkeslett_Radians', 
    'Havdybde stopp', 
    'Trekkavstand',  
    'Rundvekt',
    'Bruttotonnasje kombinert'
]

cleaned_df[all_features].describe()

,Stopposisjon lengde,Stopposisjon bredde,Stoppklokkeslett_Radians,Havdybde stopp,Trekkavstand,Rundvekt,Bruttotonnasje kombinert
count,3889.000000,3889.000000,3889.000000,3889.000000,3889.000000,3889.000000,3889.000000
mean,3.661818,61.607083,3.061212,164.693494,12260.546156,8110.606068,1913.384160
std,1.796194,1.473977,1.784911,52.827761,10511.577083,6639.417088,1093.746871
min,0.001000,57.462000,0.000000,45.000000,50.000000,10.000000,430.000000
25%,2.374000,60.911000,1.518436,121.000000,4501.000000,3394.000000,691.000000
50%,3.335000,61.768000,3.006330,157.000000,8882.000000,6711.000000,1874.000000
75%,5.318000,62.835000,4.581489,202.000000,16816.000000,10700.000000,2580.000000
max,7.576000,63.436000,6.226462,473.000000,57964.000000,55642.000000,3909.000000


In [41]:
 # Importerer verktøy for å kalkulere feil i modellen - gjennomsnittlig feil?
from sklearn.metrics import mean_absolute_error

# Lager en for-loop for å generere resultater for ulike target-values.
for predict in all_features:
    features = [f for f in all_features if f != predict]
    X = cleaned_df[features]
    y = cleaned_df[predict]

    # Definerer X og y, X er features, y er målverdien (target value)
    X = cleaned_df[features]
    y = cleaned_df[predict]

    # Splitter settet i trainsett og testsett - Da kan man sammenligne prediksjonene mot et test-sett
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=1)

    # Velger MLmodell
    fishery_model = DecisionTreeRegressor(random_state=1)

    # Trener(?) modellen på train-settet
    fishery_model.fit(X_train, y_train)

    # Prediksjoner - predikerer tar X-val verdiene og predikerer rundvekt basert på disse tallene.
    val_predictions = fishery_model.predict(X_val)

    # Sammenligner et utvalg (X-val.head()) av prediksjoner mot relaterte y_val.head()
    print(f'--------- Target value: {predict} -------')
    print(fishery_model.predict(X_val.head()))
    print(y_val.head())
   
    # Tar inn y_val verdiene og regner ut gjennomsnittlig feil mellom prediksjonene og test-verdiene(val_y)
    val_mae = mean_absolute_error(y_val, val_predictions)
    print(f'\nMean absolute error: {val_mae}\n')
   




--------- Target value: Stopposisjon lengde -------
[5.601 2.968 3.191 5.401 1.614]
1154    5.567
3740    2.985
4175    3.179
2687    5.365
185     1.586
Name: Stopposisjon lengde, dtype: float64

Mean absolute error: 0.12744344473007713

--------- Target value: Stopposisjon bredde -------
[62.733 60.485 59.655 63.11  61.68 ]
1154    62.735
3740    60.497
4175    59.546
2687    63.222
185     61.635
Name: Stopposisjon bredde, dtype: float64

Mean absolute error: 0.1537814910025707

--------- Target value: Stoppklokkeslett_Radians -------
[0.01308997 3.08923278 5.86866961 0.01745329 2.79689013]
1154    2.530727
3740    4.886922
4175    1.042834
2687    5.183628
185     1.282817
Name: Stoppklokkeslett_Radians, dtype: float64

Mean absolute error: 2.0614907106147546

--------- Target value: Havdybde stopp -------
[ 80. 113. 137. 247. 244.]
1154     89
3740    113
4175    138
2687    296
185     277
Name: Havdybde stopp, dtype: int64

Mean absolute error: 10.196658097686376

--------- Targ

Ser her at det er ganske store variasjoner i prediksjonene. Det er forsåvidt naturlig i en så uforutsigbar bransje som fiskeri.
Kan prøve med litt ulike feature og prediction values.
Ser at modellen klarer å predikere havdybde og geografisk plassering av aktivitetene ganske godt. 
Dette gir kanskje mening da fiske av trål foregår i bestemte geografiske områder, og hvor havdybden er bestemt av nettopp geografisk plassering.

Når det kommer til rundvekt, så er den ikke så treffsikker, men likevel ikke helt "off". Da fiskeri i seg selv er ganske uforutsigbart.

### Vurderinger: 
- **Radians:** Vurdere andre måter å bruke tidsaspektet i modellen

- **Rundvekt:** Dersom tidskolonnen ikke er god å bruke i modellen, kan det gi utslag på de andre prediksjonene.

- **Trekkavstand:** Denne har jeg ikke så mange tanker omkring...

ChatGPTs vurdering av resultata:
Given the descriptive statistics of your dataset, let's re-evaluate the prediction results of your Decision Tree Regressor for each feature, considering the mean, standard deviation (std), and range (min to max) of each:

### Stopposisjon Lengde & Stopposisjon Bredde
- **Stopposisjon Lengde MAE:** 0.127
- **Stopposisjon Bredde MAE:** 0.154

Given the standard deviations (1.796 for lengde, 1.474 for bredde) and the ranges (0.001 to 7.576 for lengde, 57.462 to 63.436 for bredde), the MAEs are very small in comparison. This indicates high accuracy in predictions relative to the variability of the data, showcasing the model's strong performance on these features.

### Stoppklokkeslett_Radians
- **MAE:** 2.061

Considering the standard deviation (1.785) and the range (0 to 6.226), an MAE of 2.061 is significant but not excessively large, especially given the entire range of the data. It suggests moderate accuracy, which might be due to the cyclical nature of the feature that isn't well-captured by decision trees.

### Havdybde Stopp
- **MAE:** 10.197

With a standard deviation of 52.828 and a range from 45 to 473, an MAE of 10.197 is relatively small, indicating the model's good performance in predicting the depth at which fishing stops occur.

### Trekkavstand
- **MAE:** 9786.587

Given the standard deviation (10511.577) and the range (50 to 57964), while the MAE seems high, it's somewhat proportional to the variability and range of the data. However, this does indicate the predictions could be significantly off for individual instances, suggesting room for improvement in model performance or feature engineering for this variable.

### Rundvekt
- **MAE:** 5679.35

With a substantial standard deviation (6639.417) and a wide range (10 to 55642), the MAE is somewhat high but still within the context of the data's variability. This suggests that while the model has predictive capability, there may be considerable error in specific predictions, reflecting the complexity or non-linear relationships within this feature.

### Bruttotonnasje Kombinert
- **MAE:** 850.36

Given the standard deviation (1093.747) and the range (430 to 3909), the MAE is relatively high, suggesting that predictions can be off by a significant margin for many instances. This indicates a moderate level of accuracy, potentially impacted by the limited range of this feature compared to others.

### Overall Evaluation:
- **Good Performance:** For geographical features (lengde and bredde), the model is highly accurate.
- **Moderate to Good:** For `Havdybde stopp`, the model shows good predictive capability, considering the data's variability.
- **Needs Improvement:** For `Stoppklokkeslett_Radians`, `Trekkavstand`, `Rundvekt`, and `Bruttotonnasje kombinert`, the model's predictions show significant error margins. For `Trekkavstand` and `Rundvekt`, the high MAEs are notable given the large variability in these features. This might suggest that the model is capturing some, but not all, of the underlying patterns in the data.
- **Considerations for Improvement:** Enhancements might include experimenting with different models, more sophisticated feature engineering (e.g., transforming features, adding interaction terms), or techniques to better capture the cyclical nature of time-based features. Additionally, evaluating model performance with other metrics and considering the distribution of errors could provide deeper insights into where and how the model might be improved.

In [42]:
print("Making predictions for the following 5 houses:")
print(X.head())
print("The predictions are")
print(fishery_model.predict(X.head()))

Making predictions for the following 5 houses:
   Stopposisjon lengde  Stopposisjon bredde  Stoppklokkeslett_Radians  \
0                2.134               61.278                  0.842121   
1                1.982               61.343                  2.255838   
2                1.702               61.552                  3.691371   
3                1.885               61.401                  5.628687   
4                5.664               63.401                  5.842490   

   Havdybde stopp  Trekkavstand  Rundvekt  
0             211       29124.0   30000.0  
1             158        1972.0   17000.0  
2             264       27885.0   20000.0  
3             180       16590.0    3000.0  
4             311       34192.0    1685.0  
The predictions are
[2053. 2053. 2053. 1199. 2053.]


In [43]:
from sklearn.metrics import mean_absolute_error

predicted_round_weight = fishery_model.predict(X)
mean_absolute_error(y, predicted_round_weight)

170.1157109796863

In [44]:

X = np.array(data.drop([predict], axis=1))
y = np.array(data[predict])

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Apply Standardization
min_max_scaler = MinMaxScaler()
x_train_scaled = min_max_scaler.fit_transform(x_train)
x_test_scaled = min_max_scaler.transform(x_test)
# For Min-Max Normalization, use MinMaxScaler() instead of StandardScaler()

linear = linear_model.LinearRegression()
linear.fit(x_train_scaled, y_train) # Use scaled data for training
score = linear.score(x_test_scaled, y_test) # Evaluate the model using scaled test data
print(score)

NameError: name 'data' is not defined

In [ ]:
areas = [x for x in range(0, 70)]

for area in areas:
    area_df = cleaned_df[cleaned_df['Hovedområde start (kode)'] == area]
    features = [#'Stopposisjon lengde', 
                #'Stopposisjon bredde', 
                'Stoppklokkeslett_DecimalHours', 
                'Havdybde stopp', 
                'Trekkavstand',  
                'Rundvekt',
                'Bruttotonnasje kombinert'
                ]

    data = area_df[features]
    predict = 'Rundvekt'

    X = np.array(data.drop([predict], axis=1))
    y = np.array(data[predict])

    x_train, x_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.2)

    linear = linear_model.LinearRegression()
    linear.fit(x_train, y_train)
    score = linear.score(x_test, y_test)
    print(score)

ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.